In [15]:
import heapq
import os

class BinaryTree:
    def __init__(self,value,freq):
        self.value= value
        self.freq = freq
        self.left = None
        self.right = None

    def __lt__(self,other):
        return self.freq < other.freq  # for sorting the priority queue
    def __equal__(self,other):
        return self.freq == other.freq
        
class HuffmanCode:
    def __init__(self,path):
        self.path = path
        self.__heap = []
        self.__code={}
        self.__reverse_code ={}


    def __frequency_from_test(self,text):
        freq_dict={}
        for char in text:
            if char not in freq_dict:
                freq_dict[char] = 0
            else:
                freq_dict[char]+=1
        return freq_dict
    def __Build_heap(self,frequency_dict):
        for key in frequency_dict:
            frequency = frequency_dict[key]
            binary_tree_node = BinaryTree(key,frequency)
            heapq.heappush(self.__heap , binary_tree_node)

    def __Build_binary_tree(self):
        while len(self.__heap) >1:
            binary_tree_node_1 = heapq.heappop(self.__heap)
            binary_tree_node_2 = heapq.heappop(self.__heap)
            sum_of_node = binary_tree_node_1.freq + binary_tree_node_2.freq
            newNode = BinaryTree(None, sum_of_node  )
            newNode.left = binary_tree_node_1
            newNode.right = binary_tree_node_2
            heapq.heappush(self.__heap,newNode)
        return

    def __Build_tree_code_helper(self,root,curr_bits):
        if root is None:
            return
        if root.value is not None:
            self.__code[root.value] = curr_bits
            self.__reverse_code[curr_bits] = root.value
            return
        self.__Build_tree_code_helper(root.left,curr_bits+'0')
        self.__Build_tree_code_helper(root.right,curr_bits+'1')


    def __Build_tree_code(self,):
        root = heapq.heappop(self.__heap)
        self.__Build_tree_code_helper(root, '')
        heapq.heappush(self.__heap, root)


    def __Build_encoded_text(self,text):
        encoded_text=''
        for char in text:
            encoded_text += self.__code[char]

        return encoded_text


    def __Build_padded_text(self,encoded_text):
        padding_val = 8 - len(encoded_text)%8
        for i in range(padding_val):
          encoded_text+='0'
        padded_info = "{0:08b}".format(padding_val)
        padded_text = padded_info + encoded_text
        return padded_text

    def __Build_byte_array(self,padded_text):
        byte_arr=[]
        for i in range(0, len(padded_text), 8):
          byte = padded_text[i:i+8]
          byte_arr.append(int(byte, 2))


        return byte_arr

    def compression(self):
      print("Compressing.....\n")
      filename, fileExtension = os.path.splitext(self.path)
      output_path = filename + ".bin"
      with open(self.path, "r+") as file, open(output_path, "wb") as output:
          text = file.read()
          text = text.rstrip()
          
          frequency_dict = self.__frequency_from_test(text)
          self.__Build_heap(frequency_dict)
          self.__Build_binary_tree()
          self.__Build_tree_code()  # Corrected line with parentheses
          encoded_text = self.__Build_encoded_text(text)
          padded_text = self.__Build_padded_text(encoded_text)
          byte_array = self.__Build_byte_array(padded_text)
          final_bytes = bytes(byte_array)
          output.write(final_bytes)    
      print('Compressed successfully.')
      print(output_path)
      return output_path
    def __Remove_padding(self, text):
        padded_info = text[:8]
        padding_val = int(padded_info, 2)
        text = text[8:]  # Fix the index here
        text = text[:-1 * padding_val]
        return text

    def __Decoded_text(self,text):
        current_bits=''
        decoded_text=''
        for char in text:
            current_bits+=char
            if current_bits in self.__reverse_code:
                decoded_text+=self.__reverse_code[current_bits]
                current_bits=''
        return decoded_text
            

    def decompression(self,input_path):
        filename, fileExtension = os.path.splitext(input_path)
        output_path = filename+'_decompressed'+'.txt'
        with open(input_path, "rb") as file, open(output_path, "w") as output:
            bit_string=''
            byte= file.read(1)
            while byte:
                byte=ord(byte)
                bits=bin(byte)[2:].rjust(8,'0')
                bit_string +=bits
                byte = file.read(1)
            text_wo_padding = self.__Remove_padding(bit_string)
            actual_text= self.__Decoded_text(text_wo_padding)
            output.write(actual_text)

        return output_path


#to access file and extract text from file
path = input("Enter the path of the file that you want to compress \n")
print(f"File path: {path}")
filen = HuffmanCode(path)
compressed_file =filen.compression()
filen.decompression(compressed_file)

File path: C:\Users\rajva\OneDrive\Desktop\Huffman coding\input.txt
Compressing.....

Compressed successfully.
C:\Users\rajva\OneDrive\Desktop\Huffman coding\input.bin


'C:\\Users\\rajva\\OneDrive\\Desktop\\Huffman coding\\input_decompressed.txt'